In [ ]:
# Import thư viện và khởi tạo
import ee
import geemap
import os
import numpy as np
from datetime import datetime, timedelta
import pandas as pd

# Khởi tạo Google Earth Engine với project ID
ee.Initialize(project='ee-bonglantrungmuoi')

# Cấu hình thời gian và tham số cho SMAP L4
start_date = '2022-01-01'
end_date = '2025-01-01'

# Tạo thư mục lưu dữ liệu
output_folder = './smap_l4_data'
os.makedirs(output_folder, exist_ok=True)

print(f"=== CẤU HÌNH TẢI SMAP L4 SOIL MOISTURE ===")
print(f"- Thời gian: {start_date} đến {end_date}")
print(f"- Dataset: NASA SMAP L4 Global 3-hourly 9 km")
print(f"- Mục tiêu: Độ ẩm đất cho nghiên cứu nông nghiệp")
print(f"- Thư mục lưu: {output_folder}")
print(f"- Sử dụng: geemap cho file lớn")
print(f"- Độ phân giải: 9km (gốc, không resample)")

In [ ]:
# Đọc shapefile và tạo SMAP L4 collection
print("=== TẢI SHAPEFILE VÀ TẠO SMAP L4 COLLECTION ===")

# Sử dụng shapefile Lâm Đồng từ GEE Asset
lamdong_asset = ee.FeatureCollection('projects/ee-bonglantrungmuoi/assets/lamdong')
lamdong_geometry = lamdong_asset.geometry()

# Hiển thị thông tin về shapefile
print(f"✓ Đã tải shapefile từ GEE Asset")
print(f"  - Số features: {lamdong_asset.size().getInfo()}")

# Tạo SMAP L4 collection
# Dataset: NASA SMAP L4 Global 3-hourly 9 km EASE-Grid Surface and Root Zone Soil Moisture
smap_l4_collection = (ee.ImageCollection('NASA/SMAP/SPL4SMGP/007')
                     .filterBounds(lamdong_geometry)
                     .filterDate(start_date, end_date)
                     .select(['sm_surface', 'sm_rootzone', 'sm_surface_wetness']))

# Thông tin collection
collection_size = smap_l4_collection.size().getInfo()
print(f"✓ Đã tạo SMAP L4 collection")
print(f"  - Tổng số ảnh: {collection_size}")

# Thông tin về bands
print(f"✓ Thông tin bands:")
print(f"  - sm_surface: Độ ẩm đất bề mặt (0-5cm)")
print(f"  - sm_rootzone: Độ ẩm đất tầng rễ (0-100cm)")
print(f"  - sm_surface_wetness: Độ ẩm bề mặt (0-1)")

# Thông tin ảnh đầu tiên
if collection_size > 0:
    first_image = ee.Image(smap_l4_collection.first())
    first_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd HH:mm').getInfo()
    print(f"  - Ảnh đầu tiên: {first_date}")
    
    # Thống kê theo năm
    for year in range(2022, 2025):
        year_collection = smap_l4_collection.filter(ee.Filter.calendarRange(year, year, 'year'))
        year_count = year_collection.size().getInfo()
        print(f"  - Năm {year}: {year_count} ảnh")
        
    # Thống kê theo tháng cho năm gần nhất
    print(f"  - Dữ liệu theo tháng (2024):")
    for month in range(1, 13):
        month_collection = smap_l4_collection.filter(ee.Filter.calendarRange(2024, 2024, 'year')).filter(ee.Filter.calendarRange(month, month, 'month'))
        month_count = month_collection.size().getInfo()
        if month_count > 0:
            print(f"    + Tháng {month}: {month_count} ảnh")
else:
    print("⚠️ Không tìm thấy ảnh nào!")

In [ ]:
# Functions tải dữ liệu SMAP L4
print("=== THIẾT LẬP FUNCTIONS TẢI DỮ LIỆU SMAP L4 ===")

def download_daily_composite(year, month, day, keep_original_resolution=True):
    """
    Tải composite ngày với geemap
    
    Args:
        year: Năm
        month: Tháng
        day: Ngày
        keep_original_resolution: Giữ nguyên độ phân giải gốc 9km (True)
    """
    start_date = ee.Date.fromYMD(year, month, day)
    end_date = start_date.advance(1, 'day')
    
    # Lọc ảnh theo ngày
    daily_collection = smap_l4_collection.filterDate(start_date, end_date)
    
    # Kiểm tra có dữ liệu không
    count = daily_collection.size().getInfo()
    if count == 0:
        print(f"❌ Không có dữ liệu cho {year}-{month:02d}-{day:02d}")
        return None
    
    print(f"📊 Ngày {year}-{month:02d}-{day:02d}: {count} ảnh gốc")
    
    # Tạo composite và clip
    composite = (daily_collection
                .select(['sm_surface', 'sm_rootzone', 'sm_surface_wetness'])
                .mean()  # Trung bình các lần đo trong ngày
                .clip(lamdong_geometry))
    
    # Sử dụng độ phân giải gốc 9km
    scale = 9000
    
    # Tạo tên file
    filename = f'SMAP_L4_Lamdong_{year}_{month:02d}_{day:02d}_9km.tif'
    filepath = os.path.join(output_folder, filename)
    
    try:
        print(f"🔄 Đang tải {filename}...")
        
        # Sử dụng geemap để tải file
        geemap.download_ee_image(
            image=composite,
            filename=filepath,
            scale=scale,
            region=lamdong_geometry,
            crs='EPSG:4326'
        )
        
        # Kiểm tra file size
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)  # MB
            print(f"✅ Đã lưu: {filename} ({file_size:.1f} MB)")
            return filepath
        else:
            print(f"❌ Không tạo được file: {filename}")
            return None
        
    except Exception as e:
        print(f"❌ Lỗi tải {year}-{month:02d}-{day:02d}: {str(e)}")
        return None

def download_weekly_composite(year, week, keep_original_resolution=True):
    """
    Tải composite tuần với geemap
    
    Args:
        year: Năm
        week: Tuần trong năm (1-52)
        keep_original_resolution: Giữ nguyên độ phân giải gốc 9km (True)
    """
    start_date = ee.Date.fromYMD(year, 1, 1).advance(week - 1, 'week')
    end_date = start_date.advance(1, 'week')
    
    # Lọc ảnh theo tuần
    weekly_collection = smap_l4_collection.filterDate(start_date, end_date)
    
    # Kiểm tra có dữ liệu không
    count = weekly_collection.size().getInfo()
    if count == 0:
        print(f"❌ Không có dữ liệu cho tuần {week}/{year}")
        return None
    
    print(f"📊 Tuần {week}/{year}: {count} ảnh gốc")
    
    # Tạo composite và clip
    composite = (weekly_collection
                .select(['sm_surface', 'sm_rootzone', 'sm_surface_wetness'])
                .mean()
                .clip(lamdong_geometry))
    
    # Sử dụng độ phân giải gốc 9km
    scale = 9000
    
    # Tạo tên file
    filename = f'SMAP_L4_Lamdong_{year}_W{week:02d}_9km.tif'
    filepath = os.path.join(output_folder, filename)
    
    try:
        print(f"🔄 Đang tải {filename}...")
        
        geemap.download_ee_image(
            image=composite,
            filename=filepath,
            scale=scale,
            region=lamdong_geometry,
            crs='EPSG:4326'
        )
        
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)
            print(f"✅ Đã lưu: {filename} ({file_size:.1f} MB)")
            return filepath
        else:
            print(f"❌ Không tạo được file: {filename}")
            return None
        
    except Exception as e:
        print(f"❌ Lỗi tải tuần {week}/{year}: {str(e)}")
        return None

def download_monthly_composite(year, month, keep_original_resolution=True):
    """
    Tải composite tháng với geemap
    
    Args:
        year: Năm
        month: Tháng
        keep_original_resolution: Giữ nguyên độ phân giải gốc 9km (True)
    """
    start_month = ee.Date.fromYMD(year, month, 1)
    end_month = start_month.advance(1, 'month')
    
    # Lọc ảnh theo tháng
    monthly_collection = smap_l4_collection.filterDate(start_month, end_month)
    
    # Kiểm tra có dữ liệu không
    count = monthly_collection.size().getInfo()
    if count == 0:
        print(f"❌ Không có dữ liệu cho {year}-{month:02d}")
        return None
    
    print(f"📊 Tháng {year}-{month:02d}: {count} ảnh gốc")
    
    # Tạo composite và clip
    composite = (monthly_collection
                .select(['sm_surface', 'sm_rootzone', 'sm_surface_wetness'])
                .mean()
                .clip(lamdong_geometry))
    
    # Sử dụng độ phân giải gốc 9km
    scale = 9000
    
    # Tạo tên file
    filename = f'SMAP_L4_Lamdong_{year}_{month:02d}_9km.tif'
    filepath = os.path.join(output_folder, filename)
    
    try:
        print(f"🔄 Đang tải {filename}...")
        
        geemap.download_ee_image(
            image=composite,
            filename=filepath,
            scale=scale,
            region=lamdong_geometry,
            crs='EPSG:4326'
        )
        
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)
            print(f"✅ Đã lưu: {filename} ({file_size:.1f} MB)")
            return filepath
        else:
            print(f"❌ Không tạo được file: {filename}")
            return None
        
    except Exception as e:
        print(f"❌ Lỗi tải {year}-{month:02d}: {str(e)}")
        return None

def estimate_smap_file_size(keep_original_resolution=True):
    """Ước tính kích thước file SMAP L4"""
    try:
        sample_image = ee.Image(smap_l4_collection.first())
        
        # Sử dụng độ phân giải gốc 9km
        scale = 9000
        
        pixel_count = (sample_image.select('sm_surface')
                      .reduceRegion(
                          reducer=ee.Reducer.count(),
                          geometry=lamdong_geometry,
                          scale=scale,
                          maxPixels=1e9
                      ).get('sm_surface').getInfo())
        
        # Ước tính: 3 bands × 4 bytes/pixel (Float32)
        size_mb = (pixel_count * 3 * 4) / (1024 * 1024)
        
        print(f"📏 Ước tính kích thước file SMAP L4 (độ phân giải gốc 9km):")
        print(f"   - Số pixel: {pixel_count:,}")
        print(f"   - Độ phân giải: 9km (gốc)")
        print(f"   - Số bands: 3 (sm_surface, sm_rootzone, sm_surface_wetness)")
        print(f"   - Kích thước: {size_mb:.1f} MB ({size_mb/1024:.2f} GB)")
        
        return size_mb
        
    except Exception as e:
        print(f"❌ Không thể ước tính: {str(e)}")
        return 0

def get_smap_statistics(year, month):
    """Lấy thống kê dữ liệu SMAP L4 cho tháng"""
    try:
        start_month = ee.Date.fromYMD(year, month, 1)
        end_month = start_month.advance(1, 'month')
        
        monthly_collection = smap_l4_collection.filterDate(start_month, end_month)
        
        if monthly_collection.size().getInfo() == 0:
            print(f"❌ Không có dữ liệu cho {year}-{month:02d}")
            return None
        
        # Tính thống kê
        composite = monthly_collection.mean().clip(lamdong_geometry)
        
        stats = composite.select(['sm_surface', 'sm_rootzone', 'sm_surface_wetness']).reduceRegion(
            reducer=ee.Reducer.minMax().combine(ee.Reducer.mean(), sharedInputs=True),
            geometry=lamdong_geometry,
            scale=9000,
            maxPixels=1e9
        ).getInfo()
        
        print(f"📊 Thống kê SMAP L4 cho {year}-{month:02d}:")
        for band in ['sm_surface', 'sm_rootzone', 'sm_surface_wetness']:
            if f"{band}_min" in stats:
                print(f"   - {band}:")
                print(f"     + Min: {stats[f'{band}_min']:.3f}")
                print(f"     + Max: {stats[f'{band}_max']:.3f}")
                print(f"     + Mean: {stats[f'{band}_mean']:.3f}")
        
        return stats
        
    except Exception as e:
        print(f"❌ Lỗi tính thống kê: {str(e)}")
        return None

print("✅ Đã thiết lập functions tải dữ liệu SMAP L4!")

In [ ]:
# TẢI DỮ LIỆU SMAP L4 COMPOSITE THÁNG (2022-2024)
print("=== TẢI DỮ LIỆU SMAP L4 COMPOSITE THÁNG ===")
print("Thời gian: Tháng 1/2022 → Tháng 12/2024")
print("Bands: sm_surface, sm_rootzone, sm_surface_wetness")
print("Độ phân giải: 9km (gốc, không resample)")

# Cấu hình
total_months = 3 * 12  # 3 năm × 12 tháng = 36 tháng

# Ước tính kích thước file
print(f"\n🔍 Ước tính kích thước với độ phân giải gốc 9km:")
estimated_size = estimate_smap_file_size()

print(f"📊 Tổng quan:")
print(f"   - Số tháng cần tải: {total_months} tháng")
print(f"   - Kích thước ước tính/file: {estimated_size:.1f} MB")
print(f"   - Tổng dung lượng ước tính: {(estimated_size * total_months):.1f} MB ({(estimated_size * total_months)/1024:.1f} GB)")

# Hiển thị thống kê mẫu
print(f"\n📈 Thống kê mẫu dữ liệu:")
sample_stats = get_smap_statistics(2023, 6)  # Tháng 6/2023

# Xác nhận trước khi tải
print(f"\n💡 Lưu ý quan trọng:")
print(f"   - Sử dụng độ phân giải gốc 9km (không resample)")
print(f"   - Dữ liệu cập nhật 3 lần/ngày (00:00, 08:00, 16:00 UTC)")
print(f"   - Tốc độ tải nhanh hơn do không cần resample")
print(f"   - Kích thước file nhỏ hơn so với resample")
print(f"   - Bands chính: sm_surface (0-5cm), sm_rootzone (0-100cm)")

if estimated_size > 50:  # Nếu file > 50MB
    print(f"\n⚠️  Mỗi file sẽ có kích thước {estimated_size:.0f} MB")
    confirm = input("Bạn có muốn tiếp tục tải tất cả? (y/N): ")
    if confirm.lower() != 'y':
        print("❌ Hủy bỏ quá trình tải")
        print("💡 Có thể tải từng tháng bằng: download_monthly_composite(year, month)")
        exit()

print(f"\n🚀 BẮT ĐẦU TẢI {total_months} COMPOSITE THÁNG SMAP L4 (9km)...")

downloaded_files = []
failed_months = []
count = 0

# Tải từng tháng từ 2022-2024
for year in range(2022, 2025):  # 2022, 2023, 2024
    for month in range(1, 13):  # Tháng 1-12
        count += 1
        
        print(f"\n[{count}/{total_months}] Tháng {year}-{month:02d}")
        
        try:
            # Tải dữ liệu với độ phân giải gốc
            filepath = download_monthly_composite(year, month)
            
            if filepath:
                downloaded_files.append(filepath)
                print(f"✅ Thành công: {os.path.basename(filepath)}")
                
                # Hiển thị thống kê nhanh
                get_smap_statistics(year, month)
            else:
                failed_months.append(f"{year}-{month:02d}")
                print(f"❌ Thất bại: {year}-{month:02d}")
                
        except Exception as e:
            failed_months.append(f"{year}-{month:02d}")
            print(f"❌ Lỗi {year}-{month:02d}: {str(e)}")
        
        # Hiển thị tiến trình
        progress = (count / total_months) * 100
        print(f"📈 Tiến trình: {progress:.1f}% ({count}/{total_months})")

# Tổng kết
print(f"\n{'='*60}")
print(f"🎉 HOÀN TẤT QUÁ TRÌNH TẢI SMAP L4!")
print(f"{'='*60}")

if downloaded_files:
    total_size = sum(os.path.getsize(f) for f in downloaded_files) / (1024*1024)
    
    print(f"✅ Thành công:")
    print(f"   - Đã tải: {len(downloaded_files)}/{total_months} file")
    print(f"   - Độ phân giải: 9km (gốc)")
    print(f"   - Tổng dung lượng: {total_size:.1f} MB ({total_size/1024:.1f} GB)")
    print(f"   - Thư mục: {output_folder}")
    
    # Thống kê theo năm
    for year in range(2022, 2025):
        year_files = [f for f in downloaded_files if f"_{year}_" in f]
        year_size = sum(os.path.getsize(f) for f in year_files) / (1024*1024)
        print(f"   - Năm {year}: {len(year_files)} file ({year_size:.1f} MB)")

if failed_months:
    print(f"\n❌ Thất bại ({len(failed_months)} tháng):")
    for month in failed_months:
        print(f"   - {month}")
    print(f"\n💡 Thử lại các tháng thất bại:")
    print(f"   download_monthly_composite(year, month)")

print(f"\n📋 Danh sách file đã tải:")
for i, filepath in enumerate(downloaded_files, 1):
    filename = os.path.basename(filepath)
    size = os.path.getsize(filepath) / (1024*1024)
    print(f"   {i:2d}. {filename} ({size:.1f} MB)")

print(f"\n📊 Bands trong mỗi file:")
print(f"   - Band 1: sm_surface (Độ ẩm bề mặt 0-5cm)")
print(f"   - Band 2: sm_rootzone (Độ ẩm tầng rễ 0-100cm)")
print(f"   - Band 3: sm_surface_wetness (Độ ẩm bề mặt 0-1)")

print(f"\n🔬 Sử dụng cell tiếp theo để phân tích và kiểm tra dữ liệu!")
print(f"💡 Dữ liệu độ phân giải 9km phù hợp cho nghiên cứu quy mô khu vực!")
print(f"🌍 Có thể sử dụng kỹ thuật downscaling để tăng độ phân giải nếu cần!")

In [ ]:
# KIỂM TRA VÀ PHÂN TÍCH DỮ LIỆU SMAP L4
print("=== KIỂM TRA VÀ PHÂN TÍCH DỮ LIỆU SMAP L4 ===")

def check_smap_downloaded_files():
    """Kiểm tra các file SMAP L4 đã tải với thông tin chi tiết"""
    if not os.path.exists(output_folder):
        print("❌ Thư mục dữ liệu chưa tồn tại")
        return []
    
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif')]
    
    if not files:
        print("📂 Thư mục trống - chưa có file nào")
        return []
    
    # Sắp xếp files theo tên
    files.sort()
    
    print(f"📊 Đã tải {len(files)} file SMAP L4:")
    print(f"📁 Thư mục: {output_folder}")
    
    total_size = 0
    monthly_files = []
    weekly_files = []
    daily_files = []
    
    for i, file in enumerate(files, 1):
        filepath = os.path.join(output_folder, file)
        size = os.path.getsize(filepath) / (1024*1024)  # MB
        total_size += size
        
        # Phân loại file
        if "_W" in file:
            weekly_files.append((file, size))
        elif "_" in file and len(file.split("_")) >= 5:
            if len(file.split("_")[4]) > 2:  # Ngày (DD)
                daily_files.append((file, size))
            else:  # Tháng (MM)
                monthly_files.append((file, size))
        
        print(f"   {i:2d}. {file} ({size:.1f} MB)")
    
    print(f"\n📈 Tổng quan:")
    print(f"   - Composite tháng: {len(monthly_files)} file")
    print(f"   - Composite tuần: {len(weekly_files)} file")
    print(f"   - Composite ngày: {len(daily_files)} file")
    print(f"   - Tổng dung lượng: {total_size:.1f} MB ({total_size/1024:.2f} GB)")
    
    # Thống kê theo năm
    print(f"\n📅 Thống kê theo năm:")
    for year in range(2022, 2025):
        year_files = [f for f in files if f"_{year}_" in f]
        if year_files:
            year_size = sum(os.path.getsize(os.path.join(output_folder, f)) for f in year_files) / (1024*1024)
            print(f"   - Năm {year}: {len(year_files)} file ({year_size:.1f} MB)")
    
    return files

def analyze_smap_file(filename):
    """Phân tích chi tiết file SMAP L4"""
    filepath = os.path.join(output_folder, filename)
    
    if not os.path.exists(filepath):
        print(f"❌ File không tồn tại: {filename}")
        return
    
    try:
        import rasterio
        
        with rasterio.open(filepath) as src:
            print(f"📄 Phân tích file: {filename}")
            print(f"   - Kích thước: {src.width} x {src.height} pixels")
            print(f"   - Số bands: {src.count}")
            print(f"   - CRS: {src.crs}")
            print(f"   - Bounds: {src.bounds}")
            print(f"   - Data type: {src.dtypes}")
            
            # Tên các bands
            band_names = ['sm_surface', 'sm_rootzone', 'sm_surface_wetness']
            
            # Đọc dữ liệu để thống kê
            print(f"\n📊 Thống kê dữ liệu:")
            for i, band_name in enumerate(band_names[:src.count], 1):
                band_data = src.read(i)
                
                # Loại bỏ nodata
                valid_data = band_data[band_data != src.nodata]
                
                if len(valid_data) > 0:
                    print(f"   - {band_name}:")
                    print(f"     + Min: {valid_data.min():.4f}")
                    print(f"     + Max: {valid_data.max():.4f}")
                    print(f"     + Mean: {valid_data.mean():.4f}")
                    print(f"     + Std: {valid_data.std():.4f}")
                    print(f"     + Valid pixels: {len(valid_data):,}")
                else:
                    print(f"   - {band_name}: Không có dữ liệu hợp lệ")
            
            # Tính correlation giữa các bands
            if src.count >= 2:
                print(f"\n🔗 Tương quan giữa các bands:")
                sm_surface = src.read(1).flatten()
                sm_rootzone = src.read(2).flatten()
                
                # Chỉ tính với dữ liệu hợp lệ
                valid_mask = (sm_surface != src.nodata) & (sm_rootzone != src.nodata)
                if np.sum(valid_mask) > 100:
                    corr = np.corrcoef(sm_surface[valid_mask], sm_rootzone[valid_mask])[0, 1]
                    print(f"   - sm_surface vs sm_rootzone: {corr:.3f}")
                
    except ImportError:
        print(f"💡 Cài đặt rasterio để phân tích chi tiết: pip install rasterio")
        file_size = os.path.getsize(filepath) / (1024*1024)
        print(f"📄 {filename}: {file_size:.1f} MB")
    except Exception as e:
        print(f"❌ Lỗi đọc file: {str(e)}")

def create_smap_time_series():
    """Tạo time series từ dữ liệu SMAP L4"""
    files = check_smap_downloaded_files()
    
    if not files:
        print("❌ Không có file nào để tạo time series")
        return
    
    print(f"\n📈 Tạo time series từ {len(files)} file:")
    
    # Tạo danh sách thời gian
    time_series_data = []
    
    for file in files:
        try:
            # Trích xuất thông tin thời gian từ tên file
            parts = file.split('_')
            if len(parts) >= 5:
                year = int(parts[3])
                month = int(parts[4])
                
                # Tạo datetime
                date_str = f"{year}-{month:02d}-01"
                time_series_data.append({
                    'date': date_str,
                    'year': year,
                    'month': month,
                    'filename': file
                })
        except:
            continue
    
    # Sắp xếp theo thời gian
    time_series_data.sort(key=lambda x: x['date'])
    
    print(f"✅ Tạo thành công time series với {len(time_series_data)} điểm thời gian")
    print(f"   - Từ: {time_series_data[0]['date']}")
    print(f"   - Đến: {time_series_data[-1]['date']}")
    
    # Thống kê theo tháng
    monthly_stats = {}
    for data in time_series_data:
        month = data['month']
        if month not in monthly_stats:
            monthly_stats[month] = 0
        monthly_stats[month] += 1
    
    print(f"\n📊 Thống kê theo tháng:")
    for month in range(1, 13):
        if month in monthly_stats:
            print(f"   - Tháng {month}: {monthly_stats[month]} năm")
    
    return time_series_data

def validate_smap_data():
    """Kiểm tra tính hợp lệ của dữ liệu SMAP L4"""
    files = check_smap_downloaded_files()
    
    if not files:
        print("❌ Không có file nào để kiểm tra")
        return
    
    print(f"\n🔍 Kiểm tra tính hợp lệ của {len(files)} file:")
    
    valid_files = []
    invalid_files = []
    
    for file in files:
        try:
            filepath = os.path.join(output_folder, file)
            
            # Kiểm tra kích thước file
            file_size = os.path.getsize(filepath)
            if file_size < 1024:  # < 1KB
                invalid_files.append((file, "File quá nhỏ"))
                continue
            
            # Thử đọc file
            try:
                import rasterio
                with rasterio.open(filepath) as src:
                    # Kiểm tra số bands
                    if src.count != 3:
                        invalid_files.append((file, f"Số bands không đúng: {src.count}"))
                        continue
                    
                    # Kiểm tra có dữ liệu không
                    sample = src.read(1, window=((0, 10), (0, 10)))
                    if np.all(sample == src.nodata):
                        invalid_files.append((file, "Không có dữ liệu hợp lệ"))
                        continue
                
                valid_files.append(file)
                
            except ImportError:
                # Không có rasterio, chỉ kiểm tra kích thước
                if file_size > 1024:
                    valid_files.append(file)
                else:
                    invalid_files.append((file, "File quá nhỏ"))
            
        except Exception as e:
            invalid_files.append((file, f"Lỗi: {str(e)}"))
    
    print(f"✅ File hợp lệ: {len(valid_files)}")
    print(f"❌ File không hợp lệ: {len(invalid_files)}")
    
    if invalid_files:
        print(f"\n⚠️  Các file có vấn đề:")
        for file, reason in invalid_files:
            print(f"   - {file}: {reason}")
    
    return valid_files, invalid_files

def clean_smap_folder():
    """Xóa tất cả file SMAP L4 trong thư mục"""
    if not os.path.exists(output_folder):
        print("❌ Thư mục không tồn tại")
        return
    
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif')]
    
    if not files:
        print("📂 Thư mục đã trống")
        return
    
    total_size = sum(os.path.getsize(os.path.join(output_folder, f)) for f in files) / (1024*1024)
    
    confirm = input(f"⚠️  Xóa {len(files)} file SMAP L4 ({total_size:.1f} MB)? (y/N): ")
    if confirm.lower() == 'y':
        for file in files:
            os.remove(os.path.join(output_folder, file))
        print(f"🗑️  Đã xóa {len(files)} file")
    else:
        print("❌ Hủy bỏ")

# Kiểm tra kết quả hiện tại
print("🔍 Kiểm tra trạng thái hiện tại:")
files = check_smap_downloaded_files()

if files:
    print(f"\n💡 Các chức năng có sẵn:")
    print(f"   - analyze_smap_file('filename.tif'): Phân tích chi tiết file")
    print(f"   - create_smap_time_series(): Tạo time series")
    print(f"   - validate_smap_data(): Kiểm tra tính hợp lệ")
    print(f"   - clean_smap_folder(): Xóa tất cả file")
    
    # Phân tích file đầu tiên
    if len(files) > 0:
        print(f"\n🔍 Phân tích file đầu tiên:")
        analyze_smap_file(files[0])
        
    # Tạo time series
    print(f"\n📈 Tạo time series:")
    time_series = create_smap_time_series()
    
    # Kiểm tra tính hợp lệ
    print(f"\n🔍 Kiểm tra tính hợp lệ:")
    valid_files, invalid_files = validate_smap_data()
    
    print(f"\n🎯 Tổng kết:")
    print(f"   - Tổng file: {len(files)}")
    print(f"   - File hợp lệ: {len(valid_files)}")
    print(f"   - Sẵn sàng tích hợp với Sentinel-1/2!")
    
else:
    print(f"\n💡 Chưa có dữ liệu. Chạy cell trước để tải dữ liệu SMAP L4!")

In [ ]:
# TẢI DỮ LIỆU SMAP L4 CHI TIẾT (TUẦN/NGÀY) - TÙY CHỌN
print("=== TẢI DỮ LIỆU SMAP L4 CHI TIẾT ===")
print("Chỉ sử dụng khi cần phân tích chi tiết hơn dữ liệu tháng")

def download_seasonal_analysis(year, season='summer', keep_original_resolution=True):
    """
    Tải dữ liệu theo mùa để phân tích biến động độ ẩm đất
    
    Args:
        year: Năm
        season: Mùa ('spring', 'summer', 'autumn', 'winter')
        keep_original_resolution: Giữ nguyên độ phân giải gốc 9km (True)
    """
    season_months = {
        'spring': [3, 4, 5],      # Mùa xuân
        'summer': [6, 7, 8],      # Mùa hè
        'autumn': [9, 10, 11],    # Mùa thu  
        'winter': [12, 1, 2]      # Mùa đông
    }
    
    if season not in season_months:
        print(f"❌ Mùa không hợp lệ: {season}")
        return []
    
    print(f"📊 Tải dữ liệu mùa {season} năm {year} (9km)")
    
    downloaded_files = []
    
    for month in season_months[season]:
        # Xử lý tháng 12 của mùa đông
        file_year = year if month != 12 or season != 'winter' else year - 1
        
        try:
            filepath = download_monthly_composite(file_year, month)
            if filepath:
                downloaded_files.append(filepath)
                print(f"✅ Tháng {month}: {os.path.basename(filepath)}")
            else:
                print(f"❌ Tháng {month}: Thất bại")
        except Exception as e:
            print(f"❌ Tháng {month}: {str(e)}")
    
    print(f"📈 Kết quả: {len(downloaded_files)}/{len(season_months[season])} file")
    return downloaded_files

def download_crop_season_data(year, crop_type='rice', keep_original_resolution=True):
    """
    Tải dữ liệu theo mùa vụ nông nghiệp
    
    Args:
        year: Năm
        crop_type: Loại cây trồng ('rice', 'coffee', 'vegetables')
        keep_original_resolution: Giữ nguyên độ phân giải gốc 9km (True)
    """
    crop_seasons = {
        'rice': {
            'spring': [1, 2, 3, 4],      # Vụ xuân
            'summer': [5, 6, 7, 8],      # Vụ hè
            'autumn': [9, 10, 11, 12]    # Vụ thu
        },
        'coffee': [10, 11, 12, 1, 2, 3, 4, 5],  # Mùa khô
        'vegetables': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]  # Quanh năm
    }
    
    if crop_type not in crop_seasons:
        print(f"❌ Loại cây trồng không hợp lệ: {crop_type}")
        return []
    
    print(f"🌾 Tải dữ liệu cho {crop_type} năm {year} (9km)")
    
    if crop_type == 'rice':
        print("💡 Chọn vụ lúa: spring, summer, autumn")
        season = input("Nhập vụ (spring/summer/autumn): ").lower()
        if season not in crop_seasons[crop_type]:
            print("❌ Vụ không hợp lệ")
            return []
        months = crop_seasons[crop_type][season]
    else:
        months = crop_seasons[crop_type]
    
    downloaded_files = []
    
    for month in months:
        try:
            filepath = download_monthly_composite(year, month)
            if filepath:
                downloaded_files.append(filepath)
                print(f"✅ Tháng {month}: {os.path.basename(filepath)}")
            else:
                print(f"❌ Tháng {month}: Thất bại")
        except Exception as e:
            print(f"❌ Tháng {month}: {str(e)}")
    
    print(f"📈 Kết quả: {len(downloaded_files)}/{len(months)} file")
    return downloaded_files

def download_weekly_time_series(year, month, keep_original_resolution=True):
    """
    Tải dữ liệu theo tuần cho một tháng cụ thể
    
    Args:
        year: Năm
        month: Tháng
        keep_original_resolution: Giữ nguyên độ phân giải gốc 9km (True)
    """
    print(f"📅 Tải dữ liệu tuần cho tháng {year}-{month:02d} (9km)")
    
    # Tính số tuần trong tháng
    start_date = datetime(year, month, 1)
    if month == 12:
        end_date = datetime(year + 1, 1, 1)
    else:
        end_date = datetime(year, month + 1, 1)
    
    # Tính tuần bắt đầu và kết thúc
    start_week = start_date.isocalendar()[1]
    end_week = end_date.isocalendar()[1]
    
    if end_week < start_week:  # Trường hợp năm mới
        end_week += 52
    
    downloaded_files = []
    
    for week in range(start_week, end_week + 1):
        actual_week = week if week <= 52 else week - 52
        actual_year = year if week <= 52 else year + 1
        
        try:
            filepath = download_weekly_composite(actual_year, actual_week)
            if filepath:
                downloaded_files.append(filepath)
                print(f"✅ Tuần {actual_week}: {os.path.basename(filepath)}")
            else:
                print(f"❌ Tuần {actual_week}: Thất bại")
        except Exception as e:
            print(f"❌ Tuần {actual_week}: {str(e)}")
    
    print(f"📈 Kết quả: {len(downloaded_files)} file tuần")
    return downloaded_files

def download_drought_monitoring_data(year, months=None, keep_original_resolution=True):
    """
    Tải dữ liệu cho giám sát hạn hán
    
    Args:
        year: Năm
        months: Danh sách tháng (None = tất cả)
        keep_original_resolution: Giữ nguyên độ phân giải gốc 9km (True)
    """
    if months is None:
        months = [1, 2, 3, 4, 5, 12]  # Mùa khô
    
    print(f"🔥 Tải dữ liệu giám sát hạn hán năm {year} (9km)")
    print(f"   Tháng: {months}")
    
    downloaded_files = []
    
    for month in months:
        try:
            filepath = download_monthly_composite(year, month)
            if filepath:
                downloaded_files.append(filepath)
                
                # Tính thống kê để đánh giá hạn hán
                stats = get_smap_statistics(year, month)
                if stats:
                    sm_surface_mean = stats.get('sm_surface_mean', 0)
                    print(f"✅ Tháng {month}: {os.path.basename(filepath)} (SM: {sm_surface_mean:.3f})")
                else:
                    print(f"✅ Tháng {month}: {os.path.basename(filepath)}")
            else:
                print(f"❌ Tháng {month}: Thất bại")
        except Exception as e:
            print(f"❌ Tháng {month}: {str(e)}")
    
    print(f"📈 Kết quả: {len(downloaded_files)}/{len(months)} file")
    return downloaded_files

# Hướng dẫn sử dụng
print("💡 HƯỚNG DẪN SỬ DỤNG CÁC CHỨC NĂNG CHI TIẾT:")
print("(Tất cả sử dụng độ phân giải gốc 9km)")
print()
print("🌱 Phân tích theo mùa:")
print("   download_seasonal_analysis(2023, 'summer')")
print("   download_seasonal_analysis(2023, 'winter')")
print()
print("🌾 Phân tích theo mùa vụ nông nghiệp:")
print("   download_crop_season_data(2023, 'rice')")
print("   download_crop_season_data(2023, 'coffee')")
print()
print("📅 Phân tích theo tuần:")
print("   download_weekly_time_series(2023, 6)")
print("   download_weekly_time_series(2023, 12)")
print()
print("🔥 Giám sát hạn hán:")
print("   download_drought_monitoring_data(2023, [1,2,3,4,5,12])")
print()
print("⚠️  CHÚ Ý:")
print("   - Tất cả dữ liệu sử dụng độ phân giải gốc 9km")
print("   - Tốc độ tải nhanh hơn do không resample")
print("   - Kích thước file nhỏ hơn")
print("   - Phù hợp cho nghiên cứu quy mô khu vực")
print()
print("🎯 GỢI Ý CHO DỰ ÁN:")
print("   - Sử dụng dữ liệu tháng làm cơ sở")
print("   - Tải dữ liệu tuần cho giai đoạn quan trọng")
print("   - Tải dữ liệu mùa khô cho nghiên cứu hạn hán")
print("   - Có thể downscale sau khi xử lý nếu cần")
print()

# Ví dụ tải dữ liệu mùa hè 2023
print("📊 VÍ DỤ: Tải dữ liệu mùa hè 2023 (9km)")
confirm = input("Bạn có muốn tải dữ liệu mùa hè 2023? (y/N): ")
if confirm.lower() == 'y':
    summer_files = download_seasonal_analysis(2023, 'summer')
    print(f"🎉 Đã tải {len(summer_files)} file mùa hè 2023")
else:
    print("⏭️  Bỏ qua ví dụ")